In [1]:
import sys
sys.path.append("/Users/ZRC")
sys.path

['/Users/ZRC/miniconda3/envs/tryit/lib/python36.zip',
 '/Users/ZRC/miniconda3/envs/tryit/lib/python3.6',
 '/Users/ZRC/miniconda3/envs/tryit/lib/python3.6/lib-dynload',
 '',
 '/Users/ZRC/miniconda3/envs/tryit/lib/python3.6/site-packages',
 '/Users/ZRC/miniconda3/envs/tryit/lib/python3.6/site-packages/IPython/extensions',
 '/Users/ZRC/.ipython',
 '/Users/ZRC']

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch.utils.data import RandomSampler


from torchvision import datasets
from torchvision import transforms

from torchsummary import summary

In [5]:
# Hyperparameters
batch_size = 64
num_features = 28 * 28
learning_rate = 0.1
random_seed = 7
num_epochs = 3

# Architecture
num_classes = 10
device = torch.device("cuda: 0" if torch.cuda.is_available() else "cpu")

In [6]:
data_transforms = {
    'train': transforms.Compose([
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
    ]),
}

In [7]:
train_dataset = datasets.MNIST(root = "data", 
                               train = True, 
                               transform= data_transforms["train"], 
                               download= True )

val_dataset = datasets.MNIST(root = "data", 
                               train = False, 
                               transform= data_transforms["val"], 
                               ) 

train_loader = DataLoader(dataset = train_dataset, 
                          batch_size=batch_size, 
                          shuffle= True)

val_loader = DataLoader(dataset = val_dataset,
                         batch_size=batch_size,
                         shuffle= False)

data_loader = {"train": train_loader, "val": val_loader}

## Model

In [20]:
class FullyConvNetZrc(torch.nn.Module):
    def __init__(self, num_classes):
        super(FullyConvNetZrc, self).__init__()
        
        # (w - k + 2*p)/s + 1
        
        # [1*28*28] -> [4*28*28]
        self.conv_1 = torch.nn.Conv2d(in_channels=1,
                                     out_channels=4,
                                     kernel_size=(3,3),
                                     stride=(1,1),
                                     padding=1)
        
        
        # [4*28*28] -> [4*14*14]
        self.conv_2 = torch.nn.Conv2d(in_channels=4,
                                     out_channels=4,
                                     kernel_size=(4,4),
                                     stride=(2,2),
                                     padding=1)
        
        # [4*14*14] -> [8*14*14]
        self.conv_3 = torch.nn.Conv2d(in_channels=4,
                                     out_channels=8,
                                     kernel_size=3,
                                     stride=1,
                                     padding=1)
        
        # [8*14*14] -> [8*7*7]
        self.conv_4 = torch.nn.Conv2d(in_channels=8,
                                     out_channels=8,
                                     kernel_size=3,
                                     stride=2,
                                     padding=1)
        # [8*7*7] -> [16*7*7]
        self.conv_5 = torch.nn.Conv2d(in_channels=8,
                                      out_channels=16,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=1)
        
        # [16*7*7] -> [16*4*4]
        self.conv_6 = torch.nn.Conv2d(in_channels=16,
                                      out_channels=16,
                                      kernel_size=(3, 3),
                                      stride=(2, 2),
                                      padding=1)
        
        # [16*4*4] -> [num_classes*4*4]
        self.conv_7 = torch.nn.Conv2d(in_channels=16,
                                      out_channels= num_classes,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=1)
        
        # [num_classes*4*4] -> [num_classes]
        self.out_pool = torch.nn.AdaptiveAvgPool2d(1)
        
    def forward(self, x):
        out = self.conv_1(x)
        out = F.relu(out)
        
        out = self.conv_2(out)
        out = F.relu(out)

        out = self.conv_3(out)
        out = F.relu(out)

        out = self.conv_4(out)
        out = F.relu(out)
        
        out = self.conv_5(out)
        out = F.relu(out)
        
        out = self.conv_6(out)
        out = F.relu(out)
        
        out = self.conv_7(out)
        out = F.relu(out)
        
        logits = torch.squeeze(self.out_pool(out))
        
        probas = torch.softmax(logits, dim = 1)
        
        return logits,probas

In [21]:
def init_weights(layer):
    if isinstance(layer, torch.nn.Linear):
        torch.nn.init.xavier_uniform_(layer.weight)
#         torch.nn.init.normal_(layer.weight, 0.0, 0.1)
        if layer.bias is not None:
            torch.nn.init.constant_(layer.bias.data, 0)
    elif isinstance(layer, torch.nn.Conv2d):
        torch.nn.init.kaiming_normal_(layer.weight)
        if layer.bias is not None:
            torch.nn.init.constant_(layer.bias.data, 0) 
            
model = FullyConvNetZrc(num_classes = num_classes)
model.apply(init_weights)
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [22]:
summary(model, (1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 28, 28]              40
            Conv2d-2            [-1, 4, 14, 14]             260
            Conv2d-3            [-1, 8, 14, 14]             296
            Conv2d-4              [-1, 8, 7, 7]             584
            Conv2d-5             [-1, 16, 7, 7]           1,168
            Conv2d-6             [-1, 16, 4, 4]           2,320
            Conv2d-7             [-1, 10, 4, 4]           1,450
 AdaptiveAvgPool2d-8             [-1, 10, 1, 1]               0
Total params: 6,118
Trainable params: 6,118
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.02
Estimated Total Size (MB): 0.08
----------------------------------------------------------------


In [25]:
def compute_accuracy(model, data_loader, device):
    corrected_pred, num_examples = 0,0
    model.eval()
    for features, targets in data_loader:
        features = features.to(device)
        targets = targets.to(device)
        logits, probas = model(features)
        predicted_labels = torch.argmax(probas, dim=1)
        num_examples += targets.size(0)
        corrected_pred += torch.sum((predicted_labels == targets))
    
    return float(corrected_pred) / num_examples

In [23]:
def train_model(model, data_loader, optimizer, num_epochs, metric_func, random_seed = 7):
    # Manual seed for deterministic data loader
    torch.manual_seed(random_seed)
    for epoch in range(num_epochs):
        # set training mode
        model.train() 
        for batch_idx, (features, targets) in enumerate(data_loader["train"]):
            features = features.to(device)
            targets = targets.to(device)


            ## forward pass
            logits, probas = model(features)
            loss = F.cross_entropy(logits,targets)

            # backward pass
            # clear the gradients of all tensors being optimized
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            ### Login
            if not batch_idx % 50:
                print ('Epoch: {0:03d}/{1:03d} | Batch {2:03d}/{3:03d} | Loss: {4:.2f}'.format(
                    epoch+1, num_epochs, batch_idx, 
                         len(train_dataset)//batch_size, loss))

        with torch.set_grad_enabled(False):
            print('Epoch: {0:03d}/{1:03d} training accuracy: {2:.2f}'.format(
                  epoch+1, num_epochs, 
                  metric_func(model, data_loader["train"], device)))
            
            print('Epoch: {0:03d}/{1:03d} training accuracy: {2:.2f}'.format(
                  epoch+1, num_epochs, 
                  metric_func(model, data_loader["val"], device)))

In [24]:
train_model(model, data_loader, optimizer, num_epochs, metric_func = compute_accuracy)

Epoch: 001/003 | Batch 000/937 | Loss: 2.30
Epoch: 001/003 | Batch 050/937 | Loss: 2.29
Epoch: 001/003 | Batch 100/937 | Loss: 2.25
Epoch: 001/003 | Batch 150/937 | Loss: 1.85
Epoch: 001/003 | Batch 200/937 | Loss: 1.64
Epoch: 001/003 | Batch 250/937 | Loss: 0.92
Epoch: 001/003 | Batch 300/937 | Loss: 0.42
Epoch: 001/003 | Batch 350/937 | Loss: 0.73
Epoch: 001/003 | Batch 400/937 | Loss: 0.58
Epoch: 001/003 | Batch 450/937 | Loss: 0.41
Epoch: 001/003 | Batch 500/937 | Loss: 0.21
Epoch: 001/003 | Batch 550/937 | Loss: 0.20
Epoch: 001/003 | Batch 600/937 | Loss: 0.18
Epoch: 001/003 | Batch 650/937 | Loss: 0.16
Epoch: 001/003 | Batch 700/937 | Loss: 0.11
Epoch: 001/003 | Batch 750/937 | Loss: 0.19
Epoch: 001/003 | Batch 800/937 | Loss: 0.22
Epoch: 001/003 | Batch 850/937 | Loss: 0.07
Epoch: 001/003 | Batch 900/937 | Loss: 0.24
Epoch: 001/003 training accuracy: 0.92
Epoch: 001/003 training accuracy: 0.92
Epoch: 002/003 | Batch 000/937 | Loss: 0.34
Epoch: 002/003 | Batch 050/937 | Loss: 0.0